In [29]:
import pandas
import numpy
import os

base_path = r'C:\\Users\\s_j_a\\Dropbox\\Strategies\\'
df = pandas.read_csv(os.path.join(base_path, 'Data/NYMEX_BZ-MT_prices.csv'), parse_dates=['Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)

print(df.head())

           Date  Open  High  Low   Close  Volume  Adjustment
6127 2000-01-04   NaN   NaN  NaN  2395.0     NaN           1
6126 2000-01-05   NaN   NaN  NaN  2370.0     NaN           1
6125 2000-01-06   NaN   NaN  NaN  2355.0     NaN           1
6124 2000-01-07   NaN   NaN  NaN  2335.0     NaN           1
6123 2000-01-10   NaN   NaN  NaN  2275.0     NaN           1


In [30]:
def rolling_average(srs, n):
    c = numpy.cumsum(srs, dtype=float)
    res = numpy.empty(len(srs))
    res.fill(numpy.nan)
    res[n:] = (c[n:]- c[:-n]) / n
    return res

''' a crosses b from below '''
def cross_below(a, b):
    diff = a - b
    return numpy.array( [ 1 if diff[i-1] < 0 and d > 0 else 0 for i, d in enumerate(diff)] )

In [31]:

class Trade(object):
    
    def __init__(self, date, traded_price, size, stop = None, take_profit = None, expiry = None, exit_price = None):
        self._date = date
        self._traded_price = traded_price
        self._stop = stop
        self._take_profit = take_profit
        self._expiry = expiry
        self._exit_price = exit_price
    
    def date(self):
        return self._date
    
    def traded_price(self):
         return self._traded_price
    
    def stop(self):
        return self._stop
    
    def take_profit(self):
        return self._take_profit
    
    def expiry(self):
        return self._expiry
    
    def exit_price(self):
        return self._exit_price
    
    def set_exit_price(self, price):
        self._exit_price = price
    
    @staticmethod
    def to_dict(trade):
        return dict( Date=trade.date()
                    , TradePrice=trade.traded_price()
                    , Stop=trade.stop()
                    , TakeProfit=trade.take_profit()
                    , Expiry=trade.expiry()
                    , ExitPrice=trade.exit_price() )

    

In [32]:

params = dict(short_ma = 6, long_ma = 13, ultra_long_ma = 26, stop = 0.1, take_profit = 0.05, holding_days = 25)


def strategy(params, dates, closes):
    short_ma = rolling_average(closes, params['short_ma'])
    long_ma = rolling_average(closes, params['long_ma'])
    ultra_long_ma = rolling_average(closes, params['ultra_long_ma'])
    
    crosses = cross_below(short_ma, long_ma)
    above_ultra_long = closes - ultra_long_ma
    
    live_deals = []
    dead_deals = []

    for i, (date, close, cross, above) in enumerate(zip( dates, closes, crosses, above_ultra_long)):
        if cross and above > 0:
            live_deals.append(Trade(date, close, 1, close*(1 - params['stop']), close*(1 + params['take_profit']), dates[min(i + params['holding_days'],len(dates)-1)] ))
        
        for d in live_deals:
            if close <= d.stop() or close >= d.take_profit() or date >= d.expiry():
                d.set_exit_price(close)
                dead_deals.append(d)
                live_deals.remove(d)
        
    for d in live_deals:
        d.set_exit_price(closes[-1])
        dead_deals += d

    return dead_deals
        
deals = strategy(params, df['Date'].values, df['Close'].values)
win_pct = sum([1 for d in deals if d.exit_price() - d.traded_price() > 0])/len(deals)
pandl= sum([d.exit_price() - d.traded_price() for d in deals])
losses = [d.exit_price() - d.traded_price() for d in deals if (d.exit_price() - d.traded_price()) <= 0] 
wins = [d.exit_price() - d.traded_price() for d in deals if (d.exit_price() - d.traded_price()) > 0]

p_and_l_ratio = (sum(wins)/len(wins))/(sum(numpy.abs(losses))/len(losses))

print(f'P&L {pandl:.2f}')
print(f'Deals {len(deals)}')
print(f"Win Pct {100*win_pct:.2f}")
print(f"Pts per deal {pandl/len(deals):.2f}")
print(f"P&L Ratio {p_and_l_ratio}")
print(f"Return {win_pct*(1+p_and_l_ratio)}")

P&L 20259.00
Deals 160
Win Pct 74.38
Pts per deal 126.62
P&L Ratio 0.648915610200124
Return 1.226380985086342


In [33]:
deals_df = pandas.DataFrame.from_dict([Trade.to_dict(d) for d in deals])
deals_df.to_csv(os.path.join(base_path,'Results/oil.csv'),mode='w')

In [34]:
import plotly